In [ ]:
%load_ext autoreload
%autoreload 2
import json 
import pysftp
import os
import pandas_gbq
import pandas as pd
from modules.buckets import *
from modules.reproducibility import *
from modules.sftp_operations import *
import logging

#Configure loggging
logging.basicConfig(filename='BigQuery.log', level=logging.INFO,
                   format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
logging.info('\n\n-------------New Big Query Logging Instance')

# Set the GOOGLE_APPLICATION_CREDENTIALS environment variable in order to interact, import the SFTP password from the same file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'powerschool-420113-db919282054b.json'
with open('powerschool-420113-db919282054b.json') as json_file:
    j = json.load(json_file) 
    sftp_pass = j['sftp_password']
    
# ----------------------------------------------------------
#Need to make this portion to where it assesses all files in the dir recursively. 
def main(SFTP_folder_name):

    SFTP_folder_name  = initial_schema_check(SFTP_folder_name)
    print(SFTP_folder_name)

    instance = Create(
                project_id='powerschool-420113',
                location = 'us-south1',
                bucket=f'{SFTP_folder_name}bucket-iotaschools-1',
                local_dir = fr'S:\SFTP\{SFTP_folder_name}',
                db = SFTP_folder_name,
                append_or_replace='replace',
                )
    
    instance.process()# Pass SFTP files into Bucket & then into Big Query tables


#Grab file from Clevers SFTP and replicate into PowerSchool dir
sftp_folder_name = 'powerschool'
local_dir = r'C:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\temp'
SFTP_conn(sftp_folder_name, sftp, use_pool=True)



#Populate the SFTP folders into GCP
main("powerschool")
main("EIS")

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from modules.buckets import *
from modules.reproducibility import *
from modules.sftp_operations import *
import logging

clear_logging_handlers()

try:
    #Configure loggging
    logging.basicConfig(filename='BigQuery.log', level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
    logging.info('\n\n-------------New Big Query Logging Instance')
    print('Logging setup succesful')
except Exception as e:
    print(f'Error setting up logging: {e}')


sftp = SFTPConnection(
    host='reports-sftp.clever.com',
    username='honorable-chair-4822',
    password='f7dc8316952624a1adbbe7f9',
    cnopts=cnopts,
    max_connections=5
)


sftp_folder_name = 'idm-sensitive-exports'
SFTP_conn(sftp_folder_name, sftp, use_pool=False)
sftp.close_all_connections()

#Change it to get specific file in the folder. 
#Isolate this from the process
#Create a new 


#This gets labeled as idm-sensitive-exports. 
#Need an override name. to write to the local file_path

#Only thing that needs to change is local_dir

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Logging setup succesful
Trying to download remote file: idm-sensitive-exports\google-powerschool-students-passwords.csv to local path: C:\powerschool\google-powerschool-students-passwords.csv
Trying to download remote file: idm-sensitive-exports\google-students-passwords.csv to local path: C:\powerschool\google-students-passwords.csv


In [12]:
import logging
import os

# Print current working directory to verify the location
print("Current Directory:", os.getcwd())

# Remove any existing logging handlers
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

try:
    # Configure logging
    logging.basicConfig(filename='Temp.log', level=logging.INFO,
                       format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
    logging.info('\n\n-------------New Big Query Logging Instance')

    # Flush the log to ensure it gets written
    logging.shutdown()
except Exception as e:
    print(f'Error setting up logging: {e}')


Current Directory: c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\BigQueryConn_TN
